In [1]:
%load_ext gradio

c:\Users\arpit\my_pc\repos\Skin-Lesion-Classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import requests
import numpy as np
from PIL import Image
import gradio as gr

API_URL = 'https://arpy8-skin-lesion-obj-detection-api.hf.space/process_image'
API_URL2 = 'https://arpy8-skin-lesion-classification-api.hf.space/predict'

def clf_backup(image):
    files = {'file': image}
    headers = {'accept': 'application/json',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.post(API_URL2, headers=headers, files=files)
    response.raise_for_status()
    result = response.json()
    
    return result

def process_image(image):
    if image is not None:
        try:
            files = {'file': image}
            headers = {'accept': 'application/json'}
            response = requests.post(API_URL, headers=headers, files=files)
            response.raise_for_status()
            
            if response.status_code == 200:
                annotated_image = response.json()['image']['annotated_image']
                annotated_image = Image.fromarray(np.array(annotated_image).astype(np.uint8))

                result = clf_backup(image)
                
                label = result['label']
                description = result['description']
                confidence = round(result['confidence'], 4) * 100
                link = result['link']
                
                if confidence < 75:
                    warning_message = "The image might not be of a skin lesion."
                else:
                    warning_message = None

                return annotated_image, label, description, confidence, link, warning_message
        except requests.exceptions.HTTPError as e:
            return None, None, None, None, None, f"HTTP Error: {e}"
        except Exception as e:
            return None, None, None, None, None, f"An error occurred: {response.json()}"
    else:
        return None, None, None, None, None, "Please upload an image first."

def interface(image):
    annotated_image, label, description, confidence, link, warning_message = process_image(image)
    return label, description, confidence, link, warning_message

In [8]:
%%blocks


    # results = {
    #     "Annotated Image": annotated_image,
    #     "Label": label,
    #     "Description": description,
    #     "Confidence (%)": confidence,
    #     "Link": link,
    #     "Warning": warning_message
    # }
    # return results

with gr.Blocks() as demo:
    iface = gr.Interface(
        fn=interface,
        inputs=gr.components.Image(type="filepath", label="Upload an image"),
        outputs=[
            # gr.components.Image(type="pil", label="Annotated Image"),
            gr.components.Textbox(label="Label"),
            gr.components.Textbox(label="Description"),
            gr.components.Textbox(label="Confidence (%)"),
            gr.components.Textbox(label="Link"),
            gr.components.Textbox(label="Warning")
        ],
        title="Skin Lesion Classification",
        description="Upload a skin lesion image to get its classification.",
    )

    # iface.launch()

IMPORTANT: You are using gradio version 3.50.0, however version 4.29.0 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 3.50.0, however version 4.29.0 is available, please upgrade.
--------
